In [16]:
import jax.numpy as jnp
import numpy as np
import jax
from jax import jit, grad, vmap
import time

## jax/numpy difference

In [8]:
def jax_basics():
    x_np = np.arange(5)
    x_jax = jnp.arange(5)

    print("JAX Array:", x_jax)
    #x_jax[0] = 10 # illegal operation
    y_jax = x_jax.at[0].set(10)

    print("Original JAX Array:", x_jax)
    print("Updated JAX Array:", y_jax)

jax_basics()


JAX Array: [0 1 2 3 4]
Original JAX Array: [0 1 2 3 4]
Updated JAX Array: [10  1  2  3  4]


## grad

In [16]:
def derivative_example(input_val):
    def f(x):
        return 1/(x**2)
    
    df_dx = grad(f)

    result = df_dx(input_val)

    print(f"f(x) = 1/(x^2), x = {input_val}")
    print(f"Gradient (-2/(x^3)) result: {result}")

derivative_example(2.0)

f(x) = 1/(x^2), x = 2.0
Gradient (-2/(x^3)) result: -0.25


# jit

In [13]:
def heavy_computation(x):
    return jnp.dot(x, x.T)

def jit_example():
    key = jax.random.PRNGKey(0)
    x = jax.random.normal(key, (5000,5000))

    start = time.time()
    heavy_computation(x).block_until_ready() ## make sure 
    print(f"Without JIT: {time.time() - start:.4f} seconds")

    fast_computation = jit(heavy_computation)
    fast_computation(x).block_until_ready()

    start = time.time()
    fast_computation(x).block_until_ready()
    print(f"With JIT:    {time.time() - start:.4f} seconds")

jit_example()


Without JIT: 0.0144 seconds
With JIT:    0.0115 seconds


In [14]:
import time
import jax
import jax.numpy as jnp

# --- 1. Setup Data ---
# We use a reasonably sized array
N = 2000
data_A = jax.random.normal(jax.random.PRNGKey(0), (N, N))
data_B = jax.random.normal(jax.random.PRNGKey(1), (N, N))

# --- 2. The Logic: "Loop Hell" ---
# Why this?
# Without JIT: Python dispatches op-by-op. 100 loops * 3 ops = 300 overheads.
# With JIT: XLA fuses all 300 ops into ONE single GPU/CPU kernel.
def massive_loop_operation(x):
    res = x
    # A loop that is painful for Python interpreter but easy for XLA
    for _ in range(100): 
        res = res - 0.001 * jnp.sin(res) # Element-wise math
        res = jnp.where(res > 1.0, 1.0, res) # Conditional logic
    return jnp.sum(res)

# --- 3. Benchmark Helper (With Explicit Warmup) ---
def measure_time(func, data, name):
    # 1. Warmup / Dry Run (User's logic: "Run once, disregard result")
    # This initializes any lazy allocations or caches
    _ = func(data).block_until_ready()
    
    # 2. Actual Measurement
    # Now that the system is "warm", we measure the second run
    start = time.time()
    res = func(data)
    res.block_until_ready() # CRITICAL: Wait for async computation
    end = time.time()
    
    duration = end - start
    print(f"[{name}] Time: {duration:.5f} s")
    return duration

print("==========================================")
print("   JAX BENCHMARK: No-JIT vs. JIT")
print("==========================================\n")

# --- Scenario 1: No JIT (Baseline) ---
# We explicitly check BOTH Data A and Data B without JIT
with jax.disable_jit():
    print("--- 1. Baseline (No JIT / Eager) ---")
    t_raw_A = measure_time(massive_loop_operation, data_A, "No JIT - Data A")
    t_raw_B = measure_time(massive_loop_operation, data_B, "No JIT - Data B")

# --- Scenario 2: JIT Compilation ---
print("\n--- 2. JIT Compilation Phase ---")
jit_op = jax.jit(massive_loop_operation)

# Trigger compilation. This is the "First Run" for the JIT function.
# It includes tracing + compiling optimization. It is usually slow.
start_compile = time.time()
_ = jit_op(data_A).block_until_ready()
print(f"[Compilation (Data A)] Time: {time.time() - start_compile:.5f} s")

# --- Scenario 3: After JIT (The Payoff) ---
print("\n--- 3. Optimized Execution (JIT) ---")

# Re-run Data A (Should hit cache)
# Note: measure_time func already does a 'warmup' run internally, 
# ensuring we are measuring pure execution speed.
t_jit_A = measure_time(jit_op, data_A, "JIT - Data A")

# Run Data B (Should ALSO hit cache if shapes match)
t_jit_B = measure_time(jit_op, data_B, "JIT - Data B")

# --- Final Comparison ---
print("\n==========================================")
print("             FINAL RESULTS                ")
print("==========================================")

speedup_A = t_raw_A / t_jit_A
speedup_B = t_raw_B / t_jit_B

print(f"Data A Speedup: {speedup_A:.2f}x  (Raw: {t_raw_A:.4f}s -> JIT: {t_jit_A:.4f}s)")
print(f"Data B Speedup: {speedup_B:.2f}x  (Raw: {t_raw_B:.4f}s -> JIT: {t_jit_B:.4f}s)")

if speedup_B > 5.0:
    print("\n✅ Conclusion: MASSIVE success. The loop fusion worked perfectly.")
elif speedup_B > 1.2:
    print("\n⚠️ Conclusion: Moderate success. Some overhead was removed.")
else:
    print("\n❌ Conclusion: Failure. No significant gain.")

   JAX BENCHMARK: No-JIT vs. JIT

--- 1. Baseline (No JIT / Eager) ---
[No JIT - Data A] Time: 0.10474 s
[No JIT - Data B] Time: 0.10433 s

--- 2. JIT Compilation Phase ---
[Compilation (Data A)] Time: 0.25615 s

--- 3. Optimized Execution (JIT) ---
[JIT - Data A] Time: 0.00111 s
[JIT - Data B] Time: 0.00114 s

             FINAL RESULTS                
Data A Speedup: 94.53x  (Raw: 0.1047s -> JIT: 0.0011s)
Data B Speedup: 91.89x  (Raw: 0.1043s -> JIT: 0.0011s)

✅ Conclusion: MASSIVE success. The loop fusion worked perfectly.


# vmap

In [17]:
def vmap_example():
    def vector_squared_sum(vec):
        return jnp.sum(vec ** 2)
    
    batch = jnp.array([[1, 2], [3, 4], [5, 6]])
    batch_squared_sum = vmap(vector_squared_sum)

    result = batch_squared_sum(batch)
    print("Batch result:", result)

vmap_example()

Batch result: [ 5 25 61]


In [20]:
## speed test

# --- 1. Setup Data ---
BATCH_SIZE = 5000  # Large enough to make the loop painful
DIM = 500          # Dimension of each single sample

# A batch of data: shape (5000, 500)
# This represents 5000 independent samples
data_batch = jax.random.normal(jax.random.PRNGKey(0), (BATCH_SIZE, DIM))
# A weight matrix shared across all samples
weights = jax.random.normal(jax.random.PRNGKey(1), (DIM, DIM))

# --- 2. Define the Single-Sample Operation ---
# This function is written to handle ONE sample (shape: DIM)
# It simulates a dense layer + activation + normalization
def process_single_sample(sample):
    # Matrix multiplication: (DIM) @ (DIM, DIM) -> (DIM)
    x = jnp.dot(sample, weights)
    # Activation
    x = jnp.tanh(x)
    # Some extra math (normalization-ish)
    return x / (jnp.linalg.norm(x) + 1e-6)

# --- 3. Benchmark Helper ---
def measure(func, *args, name=""):
    # Warmup
    _ = func(*args).block_until_ready()
    
    start = time.time()
    res = func(*args)
    res.block_until_ready()
    end = time.time()
    
    print(f"[{name}] Time: {end - start:.5f} s")
    return end - start

print(f"Processing Batch Size: {BATCH_SIZE}, Vector Dim: {DIM}")
print("---------------------------------------------------")

# --- Scenario 1: The Python Loop (The 'Wrong' Way) ---
# We iterate through the batch in Python and apply the function one by one.
# Even if we JIT the single function, the loop overhead kills performance.
@jax.jit
def fast_single_op(x):
    return process_single_sample(x)

def python_loop_approach(batch):
    # Using a list comprehension to loop over the 5000 samples
    return jnp.stack([fast_single_op(sample) for sample in batch])

print("Running Python Loop (Naively iterating)...")
# Note: This might take a few seconds
t_loop = measure(python_loop_approach, data_batch, name="Python Loop")


# --- Scenario 2: JAX vmap (The 'Right' Way) ---
# vmap takes the single-sample function and transforms it 
# into a function that accepts a batch.
# We also JIT the vmapped function to fuse everything into one kernel.
vmapped_op = jax.jit(jax.vmap(process_single_sample))

print("\nRunning vmap (Vectorized)...")
t_vmap = measure(vmapped_op, data_batch, name="JAX vmap")


# --- Result ---
print("\n---------------------------------------------------")
print(f"Speedup: {t_loop / t_vmap:.2f}x faster")

if t_loop / t_vmap > 10:
    print("🚀 Result: Massive speedup! vmap eliminated the loop overhead.")
else:
    print("🤔 Result: Not fast enough? Try increasing BATCH_SIZE.")

Processing Batch Size: 5000, Vector Dim: 500
---------------------------------------------------
Running Python Loop (Naively iterating)...
[Python Loop] Time: 0.92591 s

Running vmap (Vectorized)...
[JAX vmap] Time: 0.00029 s

---------------------------------------------------
Speedup: 3220.19x faster
🚀 Result: Massive speedup! vmap eliminated the loop overhead.


## MNIST Training Test

In [21]:
import jax
import jax.numpy as jnp
from flax import linen as nn
from flax.training import train_state
import optax
import numpy as np
import time

# We use PyTorch/Torchvision JUST for easy data downloading/loading.
# In a pure JAX workflow, you might use TensorFlow Datasets, but this is familiar.
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms

# --- 1. Define the CNN Architecture (Using Flax) ---
class CNN(nn.Module):
    """
    A simple Convolutional Neural Network for MNIST.
    """
    @nn.compact
    def __call__(self, x):
        # x shape: [batch, 28, 28, 1]
        x = nn.Conv(features=32, kernel_size=(3, 3))(x)
        x = nn.relu(x)
        x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
        
        x = nn.Conv(features=64, kernel_size=(3, 3))(x)
        x = nn.relu(x)
        x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
        
        # Flatten the implementation by reshaping
        x = x.reshape((x.shape[0], -1)) 
        x = nn.Dense(features=256)(x)
        x = nn.relu(x)
        x = nn.Dense(features=10)(x) # Output: 10 digits
        return x

# --- 2. Helper Functions ---
def numpy_collate(batch):
    """
    Custom collate function to convert PyTorch tensors to Numpy arrays
    compatible with JAX.
    """
    if isinstance(batch[0], np.ndarray):
        return np.stack(batch)
    elif isinstance(batch[0], (tuple,list)):
        transposed = zip(*batch)
        return [numpy_collate(samples) for samples in transposed]
    else:
        return np.array(batch)

class TrainState(train_state.TrainState):
    """
    A simple container to hold the network state (parameters + optimizer).
    """
    pass

def create_train_state(rng, learning_rate):
    """Initialize the model and optimizer."""
    cnn = CNN()
    # Initialize parameters with a dummy input (shape: [1, 28, 28, 1])
    params = cnn.init(rng, jnp.ones([1, 28, 28, 1]))['params']
    tx = optax.adam(learning_rate)
    return TrainState.create(apply_fn=cnn.apply, params=params, tx=tx)

# --- 3. The Core JAX Training Step (The "Magic" Part) ---
# @jax.jit tells JAX to compile this ENTIRE function into XLA code.
@jax.jit
def train_step(state, batch_images, batch_labels):
    """
    Runs a single training step: Forward -> Loss -> Gradient -> Update.
    """
    def loss_fn(params):
        logits = state.apply_fn({'params': params}, batch_images)
        # Calculate Cross Entropy Loss
        loss = optax.softmax_cross_entropy_with_integer_labels(
            logits=logits, labels=batch_labels
        ).mean()
        return loss, logits

    # Get gradients using JAX's auto-differentiation
    grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
    (loss, logits), grads = grad_fn(state.params)
    
    # Update the state (parameters) using the optimizer
    state = state.apply_gradients(grads=grads)
    
    # Calculate simple accuracy
    metrics = {
        'loss': loss,
        'accuracy': jnp.mean(jnp.argmax(logits, -1) == batch_labels)
    }
    return state, metrics

# --- 4. Main Execution Loop ---
def run_training():
    # Hyperparameters
    BATCH_SIZE = 128
    LEARNING_RATE = 0.001
    EPOCHS = 3
    
    # Load Data (Using Torchvision, converted to Numpy)
    # Transforming to tensor then numpy array, adding channel dimension
    transform = transforms.Compose([
        transforms.ToTensor(),
        lambda x: np.array(x).transpose(1, 2, 0) # Format: (H, W, C) -> (28, 28, 1)
    ])
    
    train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, 
                              shuffle=True, collate_fn=numpy_collate, drop_last=True)

    # Initialize State
    rng = jax.random.PRNGKey(0)
    state = create_train_state(rng, LEARNING_RATE)
    
    print("\nStarting Training...")
    print("Notice the time difference between the VERY first step and the rest.")
    print("-" * 60)

    for epoch in range(EPOCHS):
        epoch_start = time.time()
        
        for i, (batch_images, batch_labels) in enumerate(train_loader):
            step_start = time.time()
            
            # --- THE MOMENT OF TRUTH ---
            # The first time this is called, JAX compiles the graph (slow).
            # Subsequent calls reuse the compiled kernel (fast).
            state, metrics = train_step(state, batch_images, batch_labels)
            
            # Use block_until_ready() to get accurate timing for benchmarks
            metrics['loss'].block_until_ready() 
            step_end = time.time()
            
            if epoch == 0 and i < 3:
                # Log the first few steps explicitly to show compilation hit
                print(f"Epoch {epoch+1}, Step {i+1} Time: {step_end - step_start:.4f}s "
                      f"{'(COMPILING...)' if i==0 else '(Running Cached)'} | "
                      f"Loss: {metrics['loss']:.4f}")
            elif i % 100 == 0:
                print(f"Epoch {epoch+1}, Step {i+1} Time: {step_end - step_start:.4f}s | "
                      f"Acc: {metrics['accuracy']:.2%}")
        
        print(f"Epoch {epoch+1} finished in {time.time() - epoch_start:.2f}s")
        print("-" * 60)

if __name__ == "__main__":
    run_training()

100%|██████████| 9.91M/9.91M [00:01<00:00, 6.62MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 262kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 2.61MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 20.0MB/s]



Starting Training...
Notice the time difference between the VERY first step and the rest.
------------------------------------------------------------


/tmp/ipykernel_36391/3567236049.py:106: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  lambda x: np.array(x).transpose(1, 2, 0) # Format: (H, W, C) -> (28, 28, 1)


Epoch 1, Step 1 Time: 3.5787s (COMPILING...) | Loss: 2.3111
Epoch 1, Step 2 Time: 0.0027s (Running Cached) | Loss: 2.1591
Epoch 1, Step 3 Time: 0.0022s (Running Cached) | Loss: 2.0437
Epoch 1, Step 101 Time: 0.0023s | Acc: 93.75%
Epoch 1, Step 201 Time: 0.0022s | Acc: 92.97%
Epoch 1, Step 301 Time: 0.0023s | Acc: 96.88%
Epoch 1, Step 401 Time: 0.0022s | Acc: 98.44%
Epoch 1 finished in 6.56s
------------------------------------------------------------
Epoch 2, Step 1 Time: 0.0023s | Acc: 99.22%
Epoch 2, Step 101 Time: 0.0022s | Acc: 98.44%
Epoch 2, Step 201 Time: 0.0022s | Acc: 98.44%
Epoch 2, Step 301 Time: 0.0023s | Acc: 97.66%
Epoch 2, Step 401 Time: 0.0022s | Acc: 98.44%
Epoch 2 finished in 2.97s
------------------------------------------------------------
Epoch 3, Step 1 Time: 0.0023s | Acc: 98.44%
Epoch 3, Step 101 Time: 0.0023s | Acc: 99.22%
Epoch 3, Step 201 Time: 0.0023s | Acc: 99.22%
Epoch 3, Step 301 Time: 0.0022s | Acc: 100.00%
Epoch 3, Step 401 Time: 0.0023s | Acc: 100.00%
